<a href="https://colab.research.google.com/github/theoem/Data-Linkage/blob/master/Data-Linkage/Record-Linking-Fuzzy-Matching-Teodoras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Python Tools for Record Linking and Fuzzy Matching

This notebook accompanies the [article](https://pbpython.com/record-linking.html) on Practical Business Python

This notebook relies on [fuzzymatcher](https://github.com/RobinL/fuzzymatcher) and the [Python Record Linkage Toolkit](https://recordlinkage.readthedocs.io/en/latest/about.html)


In [118]:
!pip install fuzzymatcher

In [119]:
!pip install recordlinkage

In [120]:
import pandas as pd
from pathlib import Path
import fuzzymatcher
import recordlinkage

### Example using fuzzymatcher

In [121]:
acm_data = pd.read_csv(
    'https://raw.githubusercontent.com/PaoloMissier/CSC3831-2021-22/main/LINKAGE/DATASETS/ACM.csv'
)
dblp_data = pd.read_csv(
    'https://raw.githubusercontent.com/PaoloMissier/CSC3831-2021-22/main/LINKAGE/DATASETS/dblp.csv',
    encoding = 'latin_1' 
)

In [122]:
acm_data.head()

,Unnamed: 0,id,title,authors,venue,year
0,0,304586,The WASA2 object-oriented workflow management ...,"Gottfried Vossen, Mathias Weske",International Conference on Management of Data,1999
1,1,304587,A user-centered interface for querying distrib...,"Isabel F. Cruz, Kimberly M. James",International Conference on Management of Data,1999
2,2,304589,"World Wide Database-integrating the Web, CORBA...","Athman Bouguettaya, Boualem Benatallah, Lily H...",International Conference on Management of Data,1999
3,3,304590,XML-based information mediation with MIX,"Chaitan Baru, Amarnath Gupta, Bertram Lud&#228...",International Conference on Management of Data,1999
4,4,304582,The CCUBE constraint object-oriented database ...,"Alexander Brodsky, Victor E. Segal, Jia Chen, ...",International Conference on Management of Data,1999


In [123]:
dblp_data.head()

,Unnamed: 0,id,title,authors,venue,year
0,0,journals/sigmod/Mackay99,Semantic Integration of Environmental Models f...,D. Scott Mackay,SIGMOD Record,1999
1,1,conf/vldb/PoosalaI96,Estimation of Query-Result Distribution and it...,"Viswanath Poosala, Yannis E. Ioannidis",VLDB,1996
2,2,conf/vldb/PalpanasSCP02,Incremental Maintenance for Non-Distributive A...,"Themistoklis Palpanas, Richard Sidle, Hamid Pi...",VLDB,2002
3,3,conf/vldb/GardarinGT96,Cost-based Selection of Path Expression Proces...,"Zhao-Hui Tang, Georges Gardarin, Jean-Robert G...",VLDB,1996
4,4,conf/vldb/HoelS95,Benchmarking Spatial Join Operations with Spat...,"Erik G. Hoel, Hanan Samet",VLDB,1995


In [124]:
acm_data[acm_data.isnull().any(axis=1)]

,Unnamed: 0,id,title,authors,venue,year
884,884,671838,High-Performance and Scalability through Appli...,NaN,Very Large Data Bases,2000
1146,1146,673478,MineSet(tm): A System for High-End Data Mining...,NaN,Very Large Data Bases,1996
2128,2128,959075,Database principles,NaN,ACM SIGMOD Record,2003
2129,2129,959077,Book reviews,NaN,ACM SIGMOD Record,2003
2130,2130,959082,Reports,NaN,ACM SIGMOD Record,2003
2131,2131,945726,Special topic section on peer to peer data man...,NaN,ACM SIGMOD Record,2003
2132,2132,945738,Standards,NaN,ACM SIGMOD Record,2003
2133,2133,945740,Book reviews,NaN,ACM SIGMOD Record,2003
2134,2134,959063,Special section on sensor network technology a...,NaN,ACM SIGMOD Record,2003
2136,2136,758376,Author Index,NaN,Very Large Data Bases,2000


In [125]:
acm_data.dropna(subset=['authors'], inplace=True)
dblp_data.dropna(subset=['authors'], inplace=True)

In [126]:
## dataframe sizes

print("raw dataframe number of records :")
print("acm_data:  {x}".format(x=acm_data['id'].count()))
print("dblp_data:  {x}".format(x=dblp_data['id'].count()))

raw dataframe number of records :
acm_data:  2280
dblp_data:  2616


## focus on "authors"
## just pick authors starting with 'A' from both, sort and pick top 20 to visually compare


In [127]:

cond = acm_data['authors'].str.startswith('A')
print (acm_data)
acm_data.loc[cond].sort_values(by=['authors'], ascending=True)[['authors', 'title']].head(25)

      Unnamed: 0  ...  year
0              0  ...  1999
1              1  ...  1999
2              2  ...  1999
3              3  ...  1999
4              4  ...  1999
...          ...  ...   ...
2289        2289  ...  1994
2290        2290  ...  2003
2291        2291  ...  1994
2292        2292  ...  2003
2293        2293  ...  1994

[2280 rows x 6 columns]


,authors,title
1054,"A. A. Diwan, Sanjeeva Rane, S. Seshadri, S. Su...",Clustering Techniques for Minimizing External ...
1741,"A. Biliris, S. Dar, N. Gehani, H. V. Jagadish,...",ASSET: a system for supporting extended transa...
235,"A. Biliris, T. A. Funkhouser, W. O'Connell, E....",BeSS: storage support for interactive visualiz...
1565,"A. Dogac, C. Dengi, E. Kilic, G. Ozhan, F. Ozc...",METU interoperable database system
218,"A. Formica, H. D. Groger, M. Missikoff",An efficient method for checking object-orient...
1632,"A. Kemper, G. Moerkotte, K. Peithner, M. Stein...",Optimizing disjunctive queries with expensive ...
118,"A. M. Ouksel, A. Sheth",Semantic interoperability in global informatio...
2186,"A. Maedche, B. Motik, L. Stojanovic",Managing multiple and distributed ontologies o...
1529,"A. P. Buchmann, A. Deutsch, J. Zimmermann, M. ...",The REACH active OODBMS
1439,"A. Prasad Sistla, Clement T. Yu, Chengwen Liu,...",Similarity based Retrieval of Pictures Using I...


In [128]:
cond = dblp_data['authors'].str.startswith('A')
dblp_data.loc[cond].sort_values(by=['authors'], ascending=True)[['authors', 'title']].head(25)

,authors,title
207,"A. A. Diwan, S. Sudarshan, Sanjeeva Rane, S. S...",Clustering Techniques for Minimizing External ...
1279,"A. Kumaran, Jayant R. Haritsa",On the Costs of Multilingualism in Database Sy...
433,"A. Prasad Sistla, Isabel F. Cruz, Ashfaq A. Kh...",Research Activities in Database Management and...
1587,"A. Prasad Sistla, Ouri Wolfson, Yixiu Huang",Data Replication for Mobile Computers
1128,"A. Prasad Sistla, Robert H. Sloan, Yelena Yesh...",Towards a Theory of Cost Management for Digita...
1642,A. R. Dasgupta,Large Databases for Remote Sensing and GIS
703,"A. R. Hurson, Simin H. Pakzad, M. W. Bright",Automated Resolution of Semantic Heterogeneity...
2057,"Aashu Virmani, Tomasz Imielinski",DataMine - Interactive Rule Discovery System
396,"Abdelsalam Helal, Hua Li",Rainbow: Distributed Database System for Class...
2033,"Abraham Silberschatz, Christos Faloutsos, Yoss...",Modeling Skewed Distribution Using Multifracta...


## Try using the fuzzymatcher

In [129]:
dblp_data.columns

Index(['Unnamed: 0', 'id', 'title', 'authors', 'venue', 'year'], dtype='object')

In [130]:
# Columns to match on from df_left
left_on = ['title', 'authors', 'venue', 'year']

# Columns to match on from df_right
right_on = ['title', 'authors', 'venue', 'year']

In [131]:
## try link_table first
linked = fuzzymatcher.link_table(acm_data,
                                 dblp_data,
                                 left_on,
                                 right_on,
                                 left_id_col='id',
                                 right_id_col='id')

In [132]:
linked.head()

,__id_left,__id_right,match_score,match_rank,title_left,title_right,authors_left,authors_right,venue_left,venue_right,year_left,year_right
0,304586,conf/sigmod/VossenW99,2.562161,1,The WASA2 object-oriented workflow management ...,The WASA2 Object-Oriented Workflow Management ...,"Gottfried Vossen, Mathias Weske","Mathias Weske, Gottfried Vossen",International Conference on Management of Data,SIGMOD Conference,1999,1999
1,304587,conf/sigmod/CruzJ99,2.722836,1,A user-centered interface for querying distrib...,A User-Centered Interface for Querying Distrib...,"Isabel F. Cruz, Kimberly M. James","Kimberly M. James, Isabel F. Cruz",International Conference on Management of Data,SIGMOD Conference,1999,1999
2,304589,conf/sigmod/BouguettayaBH99,4.573711,1,"World Wide Database-integrating the Web, CORBA...","World Wide Database - Integrating the Web, COR...","Athman Bouguettaya, Boualem Benatallah, Lily H...","Kevin Smith, Athman Bouguettaya, Mourad Ouzzan...",International Conference on Management of Data,SIGMOD Conference,1999,1999
3,304590,conf/sigmod/BaruGLMPVC99,4.393281,1,XML-based information mediation with MIX,XML-Based Information Mediation with MIX,"Chaitan Baru, Amarnath Gupta, Bertram Lud&#228...","Yannis Papakonstantinou, Richard Marciano, Cha...",International Conference on Management of Data,SIGMOD Conference,1999,1999
4,304582,conf/sigmod/BrodskySCE99,2.968080,1,The CCUBE constraint object-oriented database ...,The CCUBE Constraint Object-Oriented Database ...,"Alexander Brodsky, Victor E. Segal, Jia Chen, ...","Alexander Brodsky, Pavel A. Exarkhopoulo, Vict...",International Conference on Management of Data,SIGMOD Conference,1999,1999


In [133]:
# Now perform the match
# It will take several minutes to run on this data set
matched_results = fuzzymatcher.fuzzy_left_join(acm_data,
                                               dblp_data,
                                               left_on,
                                               right_on,
                                               left_id_col='id',
                                               right_id_col='id')

In [134]:
matched_results[["authors_left", "authors_right"]].head()

,authors_left,authors_right
0,"Gottfried Vossen, Mathias Weske","Mathias Weske, Gottfried Vossen"
1,"Isabel F. Cruz, Kimberly M. James","Kimberly M. James, Isabel F. Cruz"
2,"Athman Bouguettaya, Boualem Benatallah, Lily H...","Kevin Smith, Athman Bouguettaya, Mourad Ouzzan..."
3,"Chaitan Baru, Amarnath Gupta, Bertram Lud&#228...","Yannis Papakonstantinou, Richard Marciano, Cha..."
4,"Alexander Brodsky, Victor E. Segal, Jia Chen, ...","Alexander Brodsky, Pavel A. Exarkhopoulo, Vict..."


In [135]:
# Reorder the columns to make viewing easier
cols = [
    "best_match_score", "title_left",	"title_right", "authors_left", 
    "authors_right", "venue_left",	"venue_right", "year_left", "year_right"
]

In [136]:
# Let's see the best matches
matched_results[cols].sort_values(by=['best_match_score'], ascending=False).head(5)

,best_match_score,title_left,title_right,authors_left,authors_right,venue_left,venue_right,year_left,year_right
28967,9.659069,TSQL2 language specification,TSQL2 Language Specification,"Richard Thomas Snodgrass, Ilsoo Ahn, Gadi Aria...","Wolfgang KÃ¤fer, Curtis E. Dyreson, Arie Segev...",ACM SIGMOD Record,SIGMOD Record,1994,1994
28303,9.585563,A TSQL2 tutorial,A TSQL2 Tutorial,"Richard Thomas Snodgrass, Ilsoo Ahn, Gad Ariav...","James Clifford, Ramez Elmasri, Michael D. Soo,...",ACM SIGMOD Record,SIGMOD Record,1994,1994
24966,9.299051,The Propel Distributed Services Platform,The Propel Distributed Services Platform,"Michael J. Carey, Steve Kirsch, Mary Roth, Ber...","Runping Qi, Kuan Yee, Brian Sterling, Nicolas ...",Very Large Data Bases,VLDB,2001,2001
34421,9.249386,A mapping mechanism to support bitmap index an...,A mapping mechanism to support bitmap index an...,"Eugene Inseok Chong, Jagannathan Srinivasan, S...","Mahesh Jagannath, Eugene Inseok Chong, Aravind...",ACM SIGMOD Record,SIGMOD Record,2003,2003
28253,8.833561,Announcement-the temporal query language TSQL2...,TSQL2 Language Specification,"Richard Thomas Snodgrass, Ilsoo Ahn, Gad Ariav...","Wolfgang KÃ¤fer, Curtis E. Dyreson, Arie Segev...",ACM SIGMOD Record,SIGMOD Record,1994,1994


In [137]:
# Now the worst matches
matched_results[cols].sort_values(by=['best_match_score'],
                                  ascending=True).head(5)

,best_match_score,title_left,title_right,authors_left,authors_right,venue_left,venue_right,year_left,year_right
19445,-1.970883,David DeWitt speaks out: on rethinking the CS ...,Gio Wiederhold Speaks Out on Moving into Acade...,Marianne Winslett,Marianne Winslett,ACM SIGMOD Record,SIGMOD Record,2002,2001
5636,-1.279158,Challenges in automating declarative business ...,Expressing Business Rules,Val Huber,Ronald G. Ross,International Conference on Management of Data,SIGMOD Conference,2000,2000
33892,-1.272599,Abstracts of invited industrial track presenta...,Objectivity Industrial Exhibit,Zachary Ives,Objectivity,International Conference on Management of Data,VLDB,2003,1998
5427,-1.258685,Going beyond personalization: rule engines at ...,Delivering High Availability for Inktomi Searc...,Eric Kintzer,Eric A. Brewer,International Conference on Management of Data,SIGMOD Conference,2000,1998
15397,-1.204926,Special system-oriented section: the best of S...,A Pattern-Based Object Calculus,Stanley Y. W. Su,"Stanley Y. W. Su, Nabil Kamel, Ping Wu",The VLDB Journal &mdash; The International Jou...,VLDB J.,1995,1994


In [138]:
# Look at the matches around 1
matched_results[cols].query("best_match_score <= 1").sort_values(
    by=['best_match_score'], ascending=False).head(10)

,best_match_score,title_left,title_right,authors_left,authors_right,venue_left,venue_right,year_left,year_right
2287,0.996159,Keynote Address,Keynote Address,Robert S. Epstein,Robert S. Epstein,International Conference on Management of Data,SIGMOD Conference,1995,1995
3443,0.995429,MQseries and CICS link for Lotus Notes,MQSeries and CICS Link for Lotus Notes,Lotus Development Corp.,?,ACM SIGMOD Record,SIGMOD Record,1996,1996
25138,0.987667,The Oracle Warehouse,The Oracle Warehouse,Gary Hallmark,Gary Hallmark,Very Large Data Bases,VLDB,1995,1995
24974,0.986689,Altruistic locking,Altruistic Locking,"Kenneth Salem, H&#233;ctor Garc&#237;a-Molina,...","Jeannie Shands, Kenneth Salem, Hector Garcia-M...",ACM Transactions on Database Systems (TODS),ACM Trans. Database Syst.,1994,1994
37745,0.978702,Power efficient data gathering and aggregation...,Power efficient data gathering and aggregation...,"H&#252;seyin &#214;zg&#252;r Tan, Ibrahim K&#2...","Ibrahim Korpeoglu, HÃ¼seyin ÃzgÃ¼r Tan",ACM SIGMOD Record,SIGMOD Record,2003,2003
36663,0.973786,On Processing XML in LDAP,On Processing XML in LDAP,"Pedro Jos&#233; Marr&#243;n, Georg Lausen","Pedro JosÃ© MarrÃ³n, Georg Lausen",Very Large Data Bases,VLDB,2001,2001
18267,0.973031,Mid-tier caching: the TimesTen approach,Mid-tier caching: the TimesTen approach,The TimesTen Team,Times-Ten Team,International Conference on Management of Data,SIGMOD Conference,2002,2002
4479,0.970899,Standard for multimedia databases,MPEG-7 Standard for Multimedia Databases,John R. Smith,John R. Smith,International Conference on Management of Data,SIGMOD Conference,2001,2001
30140,0.957410,Management of semistructured data,Foreword: Management of Semistructured Data,Dan Suciu,Dan Suciu,ACM SIGMOD Record,SIGMOD Record,1997,1997
27014,0.955514,The database group at National Technical Unive...,The Database Group at National Technical Unive...,CORPORATE National Technical Univ. of Athens,"Timos K. Sellis, Yannis Vassiliou",ACM SIGMOD Record,SIGMOD Record,1995,1995


In [139]:
matched_results[cols].query("best_match_score <= .80").sort_values(
    by=['best_match_score'], ascending=False).head(5)

,best_match_score,title_left,title_right,authors_left,authors_right,venue_left,venue_right,year_left,year_right
39340,0.789043,Cumulative Updates,Cumulative Updates,"Suryanarayana M. Sripada, Beat W&#252;thrich","Beat WÃ¼thrich, Suryanarayana M. Sripada",Very Large Data Bases,VLDB,1994,1994
3764,0.777607,Data management issues in electronic commerce:...,Guest Editor's Introduction,Asuman Dogac,Asuman Dogac,ACM SIGMOD Record,SIGMOD Record,2002,2002
35427,0.776419,Book reviews,Call for Book Reviews,Karl Aberer,Karl Aberer,ACM SIGMOD Record,SIGMOD Record,2003,2003
671,0.776227,E-commerce database issues and experience: (ta...,E-Commerce Database Issues and Experience,Anand Rajaraman,Anand Rajaraman,International Conference on Management of Data,SIGMOD Conference,1999,1999
684,0.775944,In-memory data management for consumer transac...,In-Memory Data Management for Consumer Transac...,CORPORATE TimesTen Team,Times-Ten Team,International Conference on Management of Data,SIGMOD Conference,1999,1999


### Example using Python Record Linkage Toolkit

In [140]:
acm_data = pd.read_csv(
    'https://raw.githubusercontent.com/PaoloMissier/CSC3831-2021-22/main/LINKAGE/DATASETS/ACM.csv'
)
dblp_data = pd.read_csv(
    'https://raw.githubusercontent.com/PaoloMissier/CSC3831-2021-22/main/LINKAGE/DATASETS/dblp.csv',
    encoding = 'latin_1' 
)

In [141]:
acm_data.head()

,Unnamed: 0,id,title,authors,venue,year
0,0,304586,The WASA2 object-oriented workflow management ...,"Gottfried Vossen, Mathias Weske",International Conference on Management of Data,1999
1,1,304587,A user-centered interface for querying distrib...,"Isabel F. Cruz, Kimberly M. James",International Conference on Management of Data,1999
2,2,304589,"World Wide Database-integrating the Web, CORBA...","Athman Bouguettaya, Boualem Benatallah, Lily H...",International Conference on Management of Data,1999
3,3,304590,XML-based information mediation with MIX,"Chaitan Baru, Amarnath Gupta, Bertram Lud&#228...",International Conference on Management of Data,1999
4,4,304582,The CCUBE constraint object-oriented database ...,"Alexander Brodsky, Victor E. Segal, Jia Chen, ...",International Conference on Management of Data,1999


In [142]:
dblp_data.head()

,Unnamed: 0,id,title,authors,venue,year
0,0,journals/sigmod/Mackay99,Semantic Integration of Environmental Models f...,D. Scott Mackay,SIGMOD Record,1999
1,1,conf/vldb/PoosalaI96,Estimation of Query-Result Distribution and it...,"Viswanath Poosala, Yannis E. Ioannidis",VLDB,1996
2,2,conf/vldb/PalpanasSCP02,Incremental Maintenance for Non-Distributive A...,"Themistoklis Palpanas, Richard Sidle, Hamid Pi...",VLDB,2002
3,3,conf/vldb/GardarinGT96,Cost-based Selection of Path Expression Proces...,"Zhao-Hui Tang, Georges Gardarin, Jean-Robert G...",VLDB,1996
4,4,conf/vldb/HoelS95,Benchmarking Spatial Join Operations with Spat...,"Erik G. Hoel, Hanan Samet",VLDB,1995


In [143]:
# Build the indexer
indexer = recordlinkage.Index()
# Can use full or block
#indexer.full()
#indexer.block(left_on='State', right_on='Provider State')

# Use sortedneighbor as a good option if data is not clean
indexer.sortedneighbourhood(left_on='year', right_on='year')

<Index>

In [144]:
candidates = indexer.index(acm_data, dblp_data)

In [145]:
# Let's see how many matches we want to do
print(len(candidates))

1680430


In [146]:
print(acm_data.columns)
print(dblp_data.columns)

Index(['Unnamed: 0', 'id', 'title', 'authors', 'venue', 'year'], dtype='object')
Index(['Unnamed: 0', 'id', 'title', 'authors', 'venue', 'year'], dtype='object')


In [147]:
# Takes 3 minutes using the full index.
# 14s using sorted neighbor
# 7s using blocking
compare = recordlinkage.Compare()
compare.exact('year', 'year', label='year')
compare.string('authors',
               'authors',
               threshold=0.85,
               label='authors')
compare.string('title',
               'title',
               method='jarowinkler',
               threshold=0.85,
               label='title')
compare.string('venue',
               'venue',
               method='jarowinkler',
               threshold=0.85,
               label='venue')
features = compare.compute(candidates, acm_data,
                           dblp_data)

In [148]:
features.describe()

,year,authors,title,venue
count,1.680430e+06,1.680430e+06,1.680430e+06,1680430.0
mean,3.578156e-01,7.224341e-04,1.475218e-03,0.0
std,4.793576e-01,2.686843e-02,3.838023e-02,0.0
min,0.000000e+00,0.000000e+00,0.000000e+00,0.0
25%,0.000000e+00,0.000000e+00,0.000000e+00,0.0
50%,0.000000e+00,0.000000e+00,0.000000e+00,0.0
75%,1.000000e+00,0.000000e+00,0.000000e+00,0.0
max,1.000000e+00,1.000000e+00,1.000000e+00,0.0


In [149]:
# What are the score totals?
features.sum(axis=1).value_counts().sort_index(ascending=False)

3.0        706
2.0       1781
1.0     599297
0.0    1078646
dtype: int64

In [150]:
# Get the potential matches
potential_matches = features[features.sum(axis=1) > 1].reset_index()

In [152]:
potential_matches['Score'] = potential_matches.loc[:, 'authors':'authors'].sum(axis=1)
potential_matches.head()

,level_0,level_1,year,authors,title,venue,Score
0,226,907,0,1.0,1.0,0.0,1.0
1,334,907,0,1.0,1.0,0.0,1.0
2,598,386,0,1.0,1.0,0.0,1.0
3,598,705,0,1.0,1.0,0.0,1.0
4,598,942,0,1.0,1.0,0.0,1.0


In [154]:
acm_data.loc[226,:]

Unnamed: 0                                    226
id                                         290599
title         Reminiscences on influential papers
authors                         Richard Snodgrass
venue                          ACM SIGMOD Record 
year                                         1998
Name: 226, dtype: object

In [155]:
dblp_data.loc[907,:]

Unnamed: 0                                    907
id                   journals/sigmod/Snodgrass99b
title         Reminiscences on Influential Papers
authors                      Richard T. Snodgrass
venue                               SIGMOD Record
year                                         1999
Name: 907, dtype: object

In [ ]:
# Add some convenience columns for comparing data
hospital_accounts['Acct_Name_Lookup'] = hospital_accounts[[
    'Facility Name', 'Address', 'City', 'State'
]].apply(lambda x: '_'.join(x), axis=1)

In [ ]:
hospital_reimbursement['Reimbursement_Name_Lookup'] = hospital_reimbursement[[
    'Provider Name', 'Provider Street Address', 'Provider City',
    'Provider State'
]].apply(lambda x: '_'.join(x), axis=1)

In [ ]:
reimbursement_lookup = hospital_reimbursement[['Reimbursement_Name_Lookup']].reset_index()
account_lookup = hospital_accounts[['Acct_Name_Lookup']].reset_index()

In [ ]:
account_lookup.head()

In [ ]:
reimbursement_lookup.head()

In [ ]:
account_merge = potential_matches.merge(account_lookup, how='left')

In [ ]:
account_merge.head()

In [ ]:
reimbursement_lookup.head()

In [ ]:
# Let's build a dataframe to  compare
final_merge = account_merge.merge(reimbursement_lookup, how='left')

In [ ]:
cols = [
    'Account_Num', 'Provider_Num', 'Score', 'Acct_Name_Lookup',
    'Reimbursement_Name_Lookup'
]

In [ ]:
final_merge[cols].sort_values(by=['Account_Num', 'Score'], ascending=False)

In [ ]:
# If you need to save it to Excel -
#final_merge.sort_values(by=['Account_Num', 'Score'],
#                        ascending=False).to_excel('merge_list.xlsx',
#                                                  index=False)

In [ ]:
final_merge[final_merge['Account_Num']==11035][cols]

In [ ]:
final_merge[final_merge['Account_Num']==56375][cols]

### Dedupe the data

In [ ]:
hospital_dupes = pd.read_csv(
    'https://github.com/chris1610/pbpython/raw/master/data/hospital_account_dupes.csv',
    index_col='Account_Num')

In [ ]:
hospital_dupes.head()

In [ ]:
# Deduping follows the same process, you just use 1 single dataframe
dupe_indexer = recordlinkage.Index()
dupe_indexer.sortedneighbourhood(left_on='State')
dupe_candidate_links = dupe_indexer.index(hospital_dupes)


In [ ]:
# Comparison step
compare_dupes = recordlinkage.Compare()
compare_dupes.string('City', 'City', threshold=0.85, label='City')
compare_dupes.string('Phone Number',
                     'Phone Number',
                     threshold=0.85,
                     label='Phone_Num')
compare_dupes.string('Facility Name',
                     'Facility Name',
                     threshold=0.80,
                     label='Hosp_Name')
compare_dupes.string('Address',
                     'Address',
                     threshold=0.85,
                     label='Hosp_Address')
dupe_features = compare_dupes.compute(dupe_candidate_links, hospital_dupes)

In [ ]:
dupe_features

In [ ]:
dupe_features.sum(axis=1).value_counts().sort_index(ascending=False)

In [ ]:
potential_dupes = dupe_features[dupe_features.sum(axis=1) > 2].reset_index()
potential_dupes['Score'] = potential_dupes.loc[:, 'City':'Hosp_Address'].sum(axis=1)

In [ ]:
potential_dupes.sort_values(by=['Score'], ascending=True)

In [ ]:
# Take a look at one of the potential duplicates
hospital_dupes[hospital_dupes.index.isin([51567, 41166])]